In [1]:
computer = 'win'

if computer == 'mac':
    str_chromedriver = '../../../chromedriver'
elif computer == 'win':
    str_chromedriver = '../../../chromedriver.exe'
else:
    print('？？？')
    exit(0)

# = = = = = = = = = = = = = = =

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy
import UA

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./menu.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_Selenium())
option.add_argument('user-agent=' + UA.get_User_Agent())
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

import time

# = = = = = = = = = = = = = = =

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                browser = Chrome(service=Service(str_chromedriver), options=option)

                browser.maximize_window()
                
                # = = = = = = = = = = = = = = =
                
                browser.get(url)

                time.sleep(3)
                
                # = = = = = = = = = = = = = = =
                
                if len(browser.find_elements(by=By.XPATH, value='//span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]')) != 0:
                    break
                
                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//tbody[contains(@class, "listing-inner") and contains(@class, "altrow-a-1")]')))
                list_part = browser.find_elements(by=By.XPATH, value='//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')
                
                # = = = = = = = = = = = = = = =
                
                list_part_number = [_.find_element(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]') for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                part_index = [_.text for _ in list_part_number].index(url.split(',')[-3])
                
                # = = = = = = = = = = = = = = =
                
                manufacturer = list_part[part_index].find_element(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]').text
                
                # = = = = = = = = = = = = = = =
                
                list_part_number[part_index].click()
                time.sleep(7)
                list_vehicle = browser.find_element(by=By.XPATH, value='//div[contains(@id, "buyersguidepopup-outer_c")]').find_elements(by=By.XPATH, value='./descendant::table[contains(@class, "nobmp")]')
                if len(list_vehicle) == 0:
                    vehicle = ''
                else:
                    vehicle = list_vehicle[0].text
                
                # = = = = = = = = = = = = = = =
                
                note_1 = ';'.join(_.text for _ in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]'))
                
                # = = = = = = = = = = = = = = =
                
                info = ';'.join(_.get_attribute('href') for _ in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/a[contains(@class, "ra-btn") and contains(@class, "ra-btn-moreinfo")]'))
                
                # = = = = = = = = = = = = = = =
                
                note_2 = ';'.join(_.text for _ in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row") and not(contains(@class, "moreinfo-truck"))]'))
                
                # = = = = = = = = = = = = = = =
                
                oe = ';'.join(_.get_attribute('textContent') for _ in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::span[contains(@title, "Replaces these Alternate/ OE Part Numbers")]'))
                oe = ';'.join(oe.split(', '))
                
                # = = = = = = = = = = = = = = =
                
                src = ';'.join(_.get_attribute('src') for _ in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::img[contains(@id, "inlineimg_thumb")]'))
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': url.split(',')[-3],
                                         'Manufacturer': manufacturer,
                                         'Vehicle': vehicle,
                                         'Alternate_OE_Part_Numbers': oe,
                                         'Note_1': note_1,
                                         'Note_2': note_2,
                                         'Pic': '',
                                         'Url': url,
                                         'Info': info,
                                         'Src': src}])
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                browser.quit()
                time.sleep(3)
                continue
                
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_number': url.split(',')[-3],
                                     'Url': url}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url.split(',')[-3] +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(25):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：19

5S13515  <->  [ok] - 剩余数量：0
5S15030  <->  [ok] - 剩余数量：0
5S16368  <->  [ok] - 剩余数量：0
5S6989  <->  [ok] - 剩余数量：0
5S15024  <->  [ok] - 剩余数量：0
5S12714  <->  [ok] - 剩余数量：0
5S15046  <->  [ok] - 剩余数量：0
5S16590  <->  [ok] - 剩余数量：0
5S14989  <->  [ok] - 剩余数量：0
5S15035  <->  [ok] - 剩余数量：0
5S15034  <->  [ok] - 剩余数量：0
5S15399  <->  [ok] - 剩余数量：0
5S16845  <->  [ok] - 剩余数量：0
5S16588  <->  [ok] - 剩余数量：0
5S16591  <->  [ok] - 剩余数量：0
5S12715  <->  [ok] - 剩余数量：0
5S15048  <->  [ok] - 剩余数量：0
5S13885  <->  [ok] - 剩余数量：0
5S12716  <->  [ok] - 剩余数量：0

爬虫输出中

爬虫结束


D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
